In [1]:
import numpy as np
import swiftestio as swio
import matplotlib.pyplot as plt
import timeit
import xarray as xr
from scipy.io import FortranFile
import pandas as pd

In [2]:
workingdir = '/Users/daminton/work/Projects/Swiftest/Elliott_Performance/high_high_1500_1/'

In [3]:
config_file_name = workingdir + "param.in"
config = {}
config = swio.read_swiftest_config(config_file_name)
config['BIN_OUT'] = workingdir + config['BIN_OUT']
dims  = ['time','id', 'vec']

Reading Swiftest file /Users/daminton/work/Projects/Swiftest/Elliott_Performance/high_high_1500_1/param.in


In [19]:
dsframes =[]
pl = []
tp = []
nplarr = []
ntparr = []
with FortranFile(config['BIN_OUT'], 'r') as f:
    for t, npl, plid, pvec, plab, \
           ntp, tpid, tvec, tlab in swio.swiftest_stream(f, config):
        #print(f'Time = {t[0]}')
        plframe = np.expand_dims(pvec, axis=0)
        tpframe = np.expand_dims(tvec, axis=0)
        bd = []
        
        #Create xarray DataArrays out of each body type
        if npl[0] > 0:
            plxr = xr.DataArray(plframe, dims = dims, coords = {'time' : t, 'id' : plid, 'vec' : plab})
            bd.append(plxr)
        if ntp[0] > 0:
            tpxr = xr.DataArray(tpframe, dims = dims, coords = {'time' : t, 'id' : tpid, 'vec' : tlab})
            bd.append(tpxr)
        bdxr = xr.concat(bd, dim='time')
        dsi = bdxr.to_dataset(dim='vec')
        dsi = dsi.assign(npl=npl[0])
        dsi = dsi.assign(ntp=ntp[0])
        dsframes.append(dsi)

In [20]:
ds = xr.combine_nested(dsframes, concat_dim=['time'])

In [21]:
ds

<xarray.Dataset>
Dimensions:  (id: 1595, time: 41)
Coordinates:
  * id       (id) int64 2 3 4 5 6 7 8 ... 10090 10091 10092 10093 10094 10095
  * time     (time) float64 0.0 500.0 1e+03 1.5e+03 ... 1.9e+04 1.95e+04 2e+04
Data variables:
    a        (time, id) float64 9.113e+06 1.067e+07 ... 1.113e+07 1.119e+07
    e        (time, id) float64 0.02115 0.008838 0.009927 ... 0.0227 0.01982
    inc      (time, id) float64 0.00155 0.001431 0.001883 ... 0.002669 0.002293
    capom    (time, id) float64 6.112 2.23 0.001328 ... 5.865 5.949 5.816
    omega    (time, id) float64 2.144 0.9395 4.552 5.005 ... 1.763 1.562 1.652
    capm     (time, id) float64 6.154 0.6746 3.227 0.5892 ... 3.826 3.943 3.985
    Mass     (time, id) float64 9.907e+04 3.174e+04 ... 7.501e+04 7.501e+04
    Radius   (time, id) float64 7.076e+03 4.842e+03 ... 4.377e+03 4.377e+03
    npl      (time) int32 1501 1501 1501 1501 1501 ... 1567 1567 1565 1563 1563
    ntp      (time) int32 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0